In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the dataset
df = pd.read_csv('/kaggle/input/testing/final_output_processed.csv')

# Select relevant columns
df = df[['salary', 'orgTags_SKILLS', 'position_name','position_workType']]

# Drop rows with missing values
df = df.dropna()

# Remove rows where 'orgTags_SKILLS' or 'salary' is 'NOT FOUND'
df = df[(df['orgTags_SKILLS'] != 'NOT FOUND') & (df['salary'] != 'NOT FOUND')]
df = df[(df['position_workType'] != 'NOT FOUND')]

# Remove rows where 'salary' contains letters
df = df[df['salary'].str.contains('[a-zA-Z]')==False]

# Convert 'salary' to float and then integer
df['salary'] = df['salary'].astype(float).astype(int)

# Remove rows where 'salary' is 0 or greater than 20000
df = df[(df['salary'] != 0) & (df['salary'] < 20000)]

# Combine 'orgTags_SKILLS' and 'position_name' into a single text representation
df['combined_text'] = df['orgTags_SKILLS'] + ' ' + df['position_name']+ ' ' + df['position_workType']

# TF-IDF vectorization on the combined text
tfidf_vectorizer = TfidfVectorizer()
combined_matrix = tfidf_vectorizer.fit_transform(df['combined_text'])

# Split the data
X = combined_matrix
y = df['salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict the test data
y_pred = model.predict(X_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


/tmp/ipykernel_109/458397908.py:8: DtypeWarning: Columns (13,16,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,233,234,235,236,238,239,242,244) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/kaggle/input/testing/final_output_proc

Mean Squared Error: 7510424.480354067


In [44]:
def predict_salary(input_data):
    # Preprocess the input data
    combined_text = input_data['orgTags_SKILLS'] + ' ' + input_data['position_name'] + ' ' + input_data['position_workType']

    # TF-IDF vectorization
    input_matrix = tfidf_vectorizer.transform([combined_text])

    # Predict salary
    predicted_salary = model.predict(input_matrix)

    return predicted_salary[0]


# Example input data
input_data = {
    'orgTags_SKILLS': 'python,java,ai',
    'position_name': 'data scientist',
    'position_workType': 'internship'
}

# Predict salary
predicted_salary = predict_salary(input_data)
print("Predicted Salary:", predicted_salary)


Predicted Salary: 9385.196726492151


In [47]:
import pickle

model_file_path = 'linear_regression_tfidf.pkl'

# Pickle the model
with open(model_file_path, 'wb') as file:
    pickle.dump(model, file)

print("Model pickled successfully!")

Model pickled successfully!
